
# NMT with Transformer&BPE


**To do:**

- BPE (youtokentome lib)
- nn.Transformer
- Translater en <--> ru


In [ ]:
! pip install youtokentome

We will use 
https://github.com/VKCOM/YouTokenToMe


In [ ]:
%matplotlib inline
from IPython.display import clear_output
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm.auto import tqdm
import unicodedata

import youtokentome as yttm

## Data

Same en+ru corpus. But this time, as we will represent the data not in the form of words, but in the form of sub-word parts, using Byte Pair Encoding.

In [ ]:
# Prepare data and look at it
# In addition to the dictionary, we are also interested in a set of characters
raw_alphabet = set()
alphabet = set()
def normalize(s):
    return "".join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


def preprocess(s):
    raw_alphabet.update(s)
    s = normalize(s.lower().strip())
    s = re.sub(r"[^a-zа-я?.,!]+", " ", s)
    s = re.sub(r"([.!?])", r" \1", s)
    alphabet.update(s)
    return s

pairs = []
with open('eng-rus.txt', 'r') as fin:
    for line in tqdm(fin.readlines()):
        pair = [preprocess(_) for _ in line.split('\t')]
        pairs.append(pair)

print("RAW alphabet {} symbols:".format(len(raw_alphabet)), 
      "".join(sorted(raw_alphabet)))
print("After preprocessing {} symbols: ".format(len(alphabet)), 
      "".join(sorted(alphabet)))
print("There are {} pairs".format(len(pairs)))
print(pairs[10101])

## BPE training

BPE allows us to train a dictionary of arbitrary sizes.

For example, we can make a common dictionary for English and Russian.
To do this, you need to write all available texts into one file and train BPE.

In [ ]:
lines = []
for p in pairs:
    lines += p
lines = list(set(lines))
with open("./all.txt", "w") as fout:
    for line in lines:
        fout.write(line + "\n")

! head all.txt

In [ ]:
VOCAB = 5000
bpe = yttm.BPE.train(data="./all.txt", vocab_size=VOCAB, model="enru.bpe")


## NB: is it possible to train BPE on the whole dataset

In many tasks, the question about calculating statistics on the entire dataset may arise:

<mark> if _something_ is an important feature, should this _something_ be calculates only on the train, or can we take the entire dataset with validation?</mark>

- is it possible to calculate averages over all available data for the task of time series forecasting
- is it possible to calculate word2vec on the whole dataset
- etc.

There is no simple answer, in this case BPE is not literally a model, but changing statistics can affect the composition of the dictionary.

In [ ]:
bpe.encode(lines[0])

In [ ]:
print(bpe.encode(lines[:10], output_type=yttm.OutputType.ID))
print(bpe.encode(lines[:10], output_type=yttm.OutputType.SUBWORD))

## BPE Dropout
 
 
(Article: [BPE-Dropout: Simple and Effective Subword Regularization](https://arxiv.org/abs/1910.13267))

In very large BPE dictionaries (5k tokens for two languages is a small dictionary), there is a problem: some tokens are in the dictionary, but are not found in the train data.

They can occur in real data, due to natural processes or typos. To deal with this phenomenon and simply as a regularization, you can use BPE-dropout: random re-partitioning of a string into tokens.

In [ ]:
print(bpe.encode(lines[:1], dropout_prob=0.0, output_type=yttm.OutputType.SUBWORD))
print(bpe.encode(lines[:1], dropout_prob=0.2, output_type=yttm.OutputType.SUBWORD))
print(bpe.encode(lines[:1], dropout_prob=0.5, output_type=yttm.OutputType.SUBWORD))

In [ ]:
encoded0 = [len(_) for _ in bpe.encode(lines, dropout_prob=0.0)]
encoded1 = [len(_) for _ in bpe.encode(lines, dropout_prob=0.1)]
encoded2 = [len(_) for _ in bpe.encode(lines, dropout_prob=0.5)]

sns.distplot(encoded0, kde=False, label="no do")
sns.distplot(encoded1, kde=False, label="do 0.1")
sns.distplot(encoded2, kde=False, label="do 0.5")
plt.legend()
plt.yscale('log')

In [ ]:
# it is proposed to limit the maximum string length to 100 tokens, and to use BPE_DO=0.1 for training
MAX_LENGTH = 100

##dataset

The dataset this time returns a dictionary with en and ru strings, without transformations.

collate_fn is not required for us, and we will describe the conversion to BPE inside the model class.

In [ ]:
class Pairset:
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        en, ru = self.data[item]
        return dict(en=en, ru=ru)

train_pairs, val_pairs = train_test_split(pairs, test_size=0.3)

trainset = Pairset(train_pairs)
valset = Pairset(val_pairs)

In [ ]:
trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
it = iter(trainloader)
print(next(it))

## Train Loop
It is assumed that the model has two methods:

```
model.compute_all(batch) -> Dict
model.check_translations(batch) -> None
```

In [ ]:
def train_model(model, opt, trainloader, valloader, epochs=1):
    step = 0
    logs = defaultdict(list)
    for epoch in range(epochs):
        model.train()
        for batch in tqdm(trainloader):
            details = model.compute_all(batch)
            loss = details["loss"]
            opt.zero_grad()
            loss.backward()
            opt.step()
            step += 1
            [logs[k].append(v) for k, v in details["metrics"].items()]
            
        model.eval()
        tmp = defaultdict(list)
        
        with torch.no_grad():
            for batch in tqdm(valloader):
                details = model.compute_all(batch)
                for k, v in details["metrics"].items():
                    tmp[k].append(v)
            tmp = {k: np.mean(v) for k, v in tmp.items()}
            [logs[f"val_{k}"].append(v) for k, v in tmp.items()]
            logs["step"].append(step)
            model.check_translations(batch)
        
        for key in ["loss"]:
            plt.figure()
            plt.title(key)
            plt.plot(logs[key], label="train", c='b', zorder=1)
            plt.scatter(logs["step"], logs[f"val_{key}"], label="val", c='r', zorder=10)
            plt.legend()
            plt.grid()
            plt.show()

## `nn.Transformer`

The official documentation for (nn.Transformer)[https://pytorch.org/docs/master/generated/torch.nn.Transformer.html#transformer] is rather sparse.

But the important points are:

0. You need to prepare input and output data yourself: you will need to write positional and token embeddings, as well as an output FC layer

1. nn.Transformer.forward takes on running the encoder and applying the decoder correctly.

2. The order of the axes is the same as when using RNN models (for compatibility in seq2seq tasks): `[seq_len, batch_size, dimension]`.

3. Be sure to set `src_key_padding_mask` and `tgt_key_padding_mask` to mask inaccessible tokens (in particular paddings).



It is proposed to have two special tokens: for translation into Russian and into English, with numbers `bpe.vocab_size()` and `bpe.vocab_size() + 1`.
These tokens may not be generated using the output layer, but they may be at the input.



It is proposed to write the following functions:

```
model.encode(list_of_strings) # a function that converts a string into a sequence of BPE token numbers, adds special tokens and padd to MAX_LENGTH
```


```
model.check_translations(batch) # a function that will make and display the translation for a batch with examples
```


```
model.compute_all(batch) # function for training, will run the batch, calculate the loss and return a dictionary with metrics and loss
```

In [ ]:
class VeryT(nn.Module):
    def __init__(self, bpe, bpe_dropout=0.1, hidden_size=256):
        super().__init__()
        self.hidden_size = hidden_size
        self.bpe = bpe
        self.bpe_dropout = bpe_dropout
        self.embeddings = nn.Embedding(bpe.vocab_size() + 2, hidden_size)
        self.positional_embeddings = nn.Embedding(MAX_LENGTH, hidden_size)
        self.transformer = nn.Transformer(
            d_model=hidden_size, 
            nhead=8, 
            num_encoder_layers=3, 
            num_decoder_layers=3, dim_feedforward=512)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, bpe.vocab_size()),
            nn.LogSoftmax(dim=-1),
        )
        
        
    def encode(self, lst, pre=None, post=None, seq_len=None, dropout=0.0):
        lst = [self.bpe.encode(entry, dropout_prob=dropout) for entry in lst]
        ## add tokens and paddings
        <your code>
        return lst
        
    def check_translations(self, batch):
        en, ru = [batch[key] for key in ["en", "ru"]]
        src = self.encode(en, ...)
        dst = self.encode(ru, ...)
        <your code if needed>
        src = torch.LongTensor(src)
        dst = torch.LongTensor(dst)
        with torch.no_grad():
            # generate ouput autoregressively
            for i in range(10):  # MAX_LEN - 1
                <your code>
            dst = dst.cpu().numpy()
            dst = [line.tolist() for line in dst]
            dst = self.bpe.decode(dst)
            dst = [line.replace("<PAD>", "") for line in dst]
        for line in zip(en, ru, dst):
            print("\t".join(line))
    
    def compute_all(self, batch):
        en, ru = [batch[key] for key in ["en", "ru"]]
        <formulate task>
        src = self.encode(en, ...)
        dst = self.encode(ru, ... )
        
        src = torch.LongTensor(src)
        dst = torch.LongTensor(dst)

        output = self.forward(src, dst)
        
        <compute loss>
        loss = ...
        
        return dict(
            loss=loss,
            metrics=dict(
                loss=loss.item(),
            )
        )
        
        
    def forward(self, src, dst):
        # let's a little hack:
        device = next(self.parameters()).device
        src = src.to(device)
        dst = dst.to(device)
        
        
        <build embeddings for tokens and positional>
        
        # embedded = embedded_tokens * sqrt(hidden_size) + embedded_positions
        
        <reshape properly>
        
        <build pad masks>
        src_pad_mask = src != 0
        dst_pad_mask = dst != 0
        
        output = self.transformer(src_embedded, dst_embedded, 
                                  src_key_padding_mask=src_pad_mask, 
                                  tgt_key_padding_mask=dst_pad_mask)
        <predict next token probs>
        <permute to [bs, vocab, seq_len]> 
        return output

In [ ]:
# check dimensions
model = VeryT(bpe)
with torch.no_grad():
    batch = next(it)
    model.check_translations(batch)

In [ ]:
if torch.cuda.is_available():
    device="cuda:0"
print(device)

model = VeryT(bpe)
model.to(device)
opt = optim.Adam(model.parameters(), lr=3e-4)


In [ ]:
trainloader = DataLoader(trainset, batch_size=50, shuffle=True)
valloader = DataLoader(valset, batch_size=50, shuffle=False)

train_model(model, opt, trainloader, valloader)